# Segmenting and Clustering the neighborhoods in Toronto

## <b> Question 1

In [6]:
#importing all the required libraries
import numpy as np
import pandas as pd
import requests
import sys

### Scraping the wikipedia page


In [7]:
import pip
if int(pip.__version__.split('.')[0])>9:
        from pip._internal import main
else:
        from pip import main
def install(package):
       main(['install', package])

install('BeautifulSoup4')

In [8]:
#import beautifulsoup and csv
import requests
from urllib.request import urlopen
from bs4 import BeautifulSoup
import ssl
import csv

In [9]:
#reading data from wikipedia page into dataframe
wikipedia_link='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
raw_wikipedia_page= requests.get(wikipedia_link).text

# using beautiful soup to parse the HTML code.
soup = BeautifulSoup(raw_wikipedia_page,'html.parser')

In [10]:
table = soup.find('table',{'class':'wikitable sortable'})
table_rows = table.find_all('tr')

### Creating the Dataframe

In [11]:
data = []
for row in table_rows:
    data.append([t.text.strip() for t in row.find_all('td')])
    
df = pd.DataFrame(data, columns=['PostalCode', 'Borough', 'Neighbourhood'])
df = df[~df['PostalCode'].isnull()]

In [12]:
#data is transformed into pandas dataframe
df.info()
df.shape

<class 'pandas.core.frame.DataFrame'>
Int64Index: 180 entries, 1 to 180
Data columns (total 3 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   PostalCode     180 non-null    object
 1   Borough        180 non-null    object
 2   Neighbourhood  180 non-null    object
dtypes: object(3)
memory usage: 5.6+ KB


(180, 3)

In [13]:
#only consider the assigned boroughs
import requests
from bs4 import BeautifulSoup
website_text = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = BeautifulSoup(website_text,'html.parser')

table = soup.find('table',{'class':'wikitable sortable'})
table_rows = table.find_all('tr')

data = []
for row in table_rows:
    data.append([t.text.strip() for t in row.find_all('td')])

df = pd.DataFrame(data, columns=['PostalCode', 'Borough', 'Neighbourhood'])
df = df[~df['PostalCode'].isnull()]

In [14]:
#drop the cells that have unassigned boroughs
df.drop(df[df['Borough']=="Not assigned"].index,axis=0, inplace=True)

In [15]:
#new dataframe df1 after reseting the index
df1 = df.reset_index()

In [16]:
df1.info()
df1.shape

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 103 entries, 0 to 102
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   index          103 non-null    int64 
 1   PostalCode     103 non-null    object
 2   Borough        103 non-null    object
 3   Neighbourhood  103 non-null    object
dtypes: int64(1), object(3)
memory usage: 3.3+ KB


(103, 4)

In [17]:
#grouping the dataframe on the basis of postal code
df2= df1.groupby('PostalCode').agg(lambda x: ','.join(x))
df2.info()
df2.shape

<class 'pandas.core.frame.DataFrame'>
Index: 103 entries, M1B to M9W
Data columns (total 2 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Borough        103 non-null    object
 1   Neighbourhood  103 non-null    object
dtypes: object(2)
memory usage: 2.4+ KB


(103, 2)

In [18]:
#assigning no assigned neighbourhood cells bporoughs as the neighbourhoods
df2.loc[df2['Neighbourhood']=="Not assigned",'Neighbourhood']=df2.loc[df2['Neighbourhood']=="Not assigned",'Borough']

In [19]:
df3 = df2.reset_index()

In [20]:
df3['Borough']= df3['Borough'].str.replace('nan|[{}\s]','').str.split(',').apply(set).str.join(',').str.strip(',').str.replace(",{2,}",",")

In [21]:
# pandas dataframe on the information regarding the neighbourhoods of toronto
df3.head(12)

,PostalCode,Borough,Neighbourhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"Kennedy Park, Ionview, East Birchmount Park"
7,M1L,Scarborough,"Golden Mile, Clairlea, Oakridge"
8,M1M,Scarborough,"Cliffside, Cliffcrest, Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff, Cliffside West"


In [22]:
print('dataframe created')

dataframe created


In [23]:
df3.info()
df3.shape

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 103 entries, 0 to 102
Data columns (total 3 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   PostalCode     103 non-null    object
 1   Borough        103 non-null    object
 2   Neighbourhood  103 non-null    object
dtypes: object(3)
memory usage: 2.5+ KB


(103, 3)

## Question 2

### Creating the toronto neighbourhood dataframe with latitude,longitude values.

In [24]:
conda install -c conda-forge geopy

Solving environment: done

# All requested packages already installed.


Note: you may need to restart the kernel to use updated packages.


In [60]:
#import geocoders and Nominatim
from  geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent='toronto_explorer')
#test for a location 
location = geolocator.geocode("Toronto, North York, Parkwoods")
print(location.address)
print('')
print((location.latitude, location.longitude))
print('')
print(location.raw)

Parkwoods Village Drive, Parkway East, Don Valley East, North York, Toronto, Golden Horseshoe, Ontario, M3A 2X2, Canada

(43.7587999, -79.3201966)

{'place_id': 124974741, 'licence': 'Data © OpenStreetMap contributors, ODbL 1.0. https://osm.org/copyright', 'osm_type': 'way', 'osm_id': 160406961, 'boundingbox': ['43.7576231', '43.761106', '-79.3239088', '-79.316215'], 'lat': '43.7587999', 'lon': '-79.3201966', 'display_name': 'Parkwoods Village Drive, Parkway East, Don Valley East, North York, Toronto, Golden Horseshoe, Ontario, M3A 2X2, Canada', 'class': 'highway', 'type': 'secondary', 'importance': 0.51}


In [61]:
import pandas as pd
df_geopy = pd.DataFrame({'PostalCode': ['M3A', 'M4A', 'M5A'],
                         'Borough': ['North York', 'North York', 'Downtown Toronto'],
                         'Neighbourhood': ['Parkwoods', 'Victoria Village', 'Harbourfront'],})
df_geo = df3

In [62]:
#add a column callled adress that is the combination of postalcode,borough,neighbourhood
geolocator = Nominatim(user_agent='toronto_explorer')

df_geo['address'] = df3[['PostalCode', 'Borough', 'Neighbourhood']].apply(lambda x: ', '.join(x), axis=1 )
df_geo.head()

,PostalCode,Borough,Neighbourhood,address,Coordinates,latitude,longitude
0,M1B,Scarborough,"Malvern, Rouge","M1B, Scarborough, Malvern, Rouge",None,NaN,NaN
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek","M1C, Scarborough, Rouge Hill, Port Union, High...",None,NaN,NaN
2,M1E,Scarborough,"Guildwood, Morningside, West Hill","M1E, Scarborough, Guildwood, Morningside, West...",None,NaN,NaN
3,M1G,Scarborough,Woburn,"M1G, Scarborough, Woburn","(Woburn, Scarborough—Guildwood, Scarborough, T...",43.759824,-79.225291
4,M1H,Scarborough,Cedarbrae,"M1H, Scarborough, Cedarbrae",None,NaN,NaN


In [63]:
df_geo = df3

In [65]:
#drop the boroughs that are not assigned
df_geo.drop(df_geo[df_geo['Borough']=="Notassigned"].index,axis=0, inplace=True)
df_geo.info()
df_geo.shape

<class 'pandas.core.frame.DataFrame'>
Int64Index: 103 entries, 0 to 102
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   PostalCode     103 non-null    object 
 1   Borough        103 non-null    object 
 2   Neighbourhood  103 non-null    object 
 3   address        103 non-null    object 
 4   Coordinates    3 non-null      object 
 5   latitude       3 non-null      float64
 6   longitude      3 non-null      float64
dtypes: float64(2), object(5)
memory usage: 6.4+ KB


(103, 7)

In [68]:
#saving the df_geo dataframe values to "geo" csv file
df_geo.to_csv('geo.csv')

In [69]:
#opening geopy.csv file for reading
df3.to_csv('geopy.csv')
import csv

with open('geopy.csv') as csvfile:
     reader = csv.DictReader(csvfile)

### Creating a CSV file

In [70]:
#creating a csv file for saving postalcode,neighbourhood,borough,latitude,longitude data
import numpy as np
import csv


PostalCode = None
Borough = None
Neighbourhood = None
latData = None
longData = None

LAT_Woburn = 43.7598243
LONG_Woburn = -79.2252908
LAT_Malvern = 43.8091955
LONG_Malvern = -79.2217008
LAT_Highland_Creek = 43.7901172
LONG_Highland_Creek = -79.1733344
LAT_Morningside = 43.7826012
LONG_Morningside = -79.2049579

 
PostalCode = np.array(['M1H','M1B','M1C','M1G '])
Borough = np.array(['Scarborough','Scarborough','Scarborough','Scarborough'])
Neighbourhood = np.array(['Woburn','Malvern','Highland_Creek','Morningside'])
latData = np.array([43.7598243,43.8091955, 43.7901172 , 43.7826012])
longData = np.array([-79.2252908,-79.2217008,-79.1733344, -79.2049579 ])

with open('data.csv', 'w') as file:
    writer = csv.writer(file, delimiter=',')
    writer.writerow('ABXYZ')
    for a,b,x,y,z in np.nditer([ PostalCode.T, Borough.T, Neighbourhood.T, latData.T, longData.T], order='C'):
        writer.writerow([a,b,x,y,z])    

In [89]:
#Reading the contents of the csv file
import csv

with open('data.csv') as csvfile:
     reader = csv.DictReader(csvfile)
     for row in reader:
         print(row['A'], row['B'],row['X'], row['Y'], row['Z'])

M1H Scarborough Woburn 43.7598243 -79.2252908
M1B Scarborough Malvern 43.8091955 -79.2217008
M1C Scarborough Highland_Creek 43.7901172 -79.1733344
M1G  Scarborough Morningside 43.7826012 -79.2049579


In [73]:
pd.read_csv('data.csv') 

,A,B,X,Y,Z
0,M1H,Scarborough,Woburn,43.759824,-79.225291
1,M1B,Scarborough,Malvern,43.809196,-79.221701
2,M1C,Scarborough,Highland_Creek,43.790117,-79.173334
3,M1G,Scarborough,Morningside,43.782601,-79.204958


### Creating Dataframes

In [38]:
df_geopy=df3
type(df_geopy)

pandas.core.frame.DataFrame

In [74]:
#adding address column to the geopy dataframe
df_geopy['address']=df_geopy['PostalCode'] + ',' + df_geopy['Borough'] + ','+ df_geopy['Neighbourhood']
df_geopy.head()

,PostalCode,Borough,Neighbourhood,address
0,M3A,North York,Parkwoods,"M3A,North York,Parkwoods"
1,M4A,North York,Victoria Village,"M4A,North York,Victoria Village"
2,M5A,Downtown Toronto,Harbourfront,"M5A,Downtown Toronto,Harbourfront"


In [75]:
#adding the coordinates column to the dataframe
df_geopy['Coordinates'] =df_geopy['address'].apply(Nominatim(user_agent='toronto_explorer').geocode)
df_geopy.head()

,PostalCode,Borough,Neighbourhood,address,Coordinates
0,M3A,North York,Parkwoods,"M3A,North York,Parkwoods","(Parkwoods Village Drive, Parkway East, Don Va..."
1,M4A,North York,Victoria Village,"M4A,North York,Victoria Village",None
2,M5A,Downtown Toronto,Harbourfront,"M5A,Downtown Toronto,Harbourfront",None


In [79]:
#Adding latitude,Longitude values to the dataframe
df_geopy['latitude']=df_geopy['Coordinates'].apply(lambda x: x.latitude if x !=None else None)
df_geopy['longitude']=df_geopy['Coordinates'].apply(lambda x: x.longitude if x !=None else None)
df_geopy.head()

,PostalCode,Borough,Neighbourhood,address,Coordinates,latitude,longitude
0,M3A,North York,Parkwoods,"M3A,North York,Parkwoods","(Parkwoods Village Drive, Parkway East, Don Va...",43.761224,-79.323986
1,M4A,North York,Victoria Village,"M4A,North York,Victoria Village",None,NaN,NaN
2,M5A,Downtown Toronto,Harbourfront,"M5A,Downtown Toronto,Harbourfront",None,NaN,NaN


In [80]:
df_geopy.to_csv('geo_loc_py.csv')

In [82]:
#Saving the data n geopy csv file into data1 datafrrame
data1 = pd.read_csv("geopy.csv") 
data1.head()

,Unnamed: 0,PostalCode,Borough,Neighbourhood,address,Coordinates,latitude,longitude
0,0,M1B,Scarborough,"Malvern, Rouge","M1B, Scarborough, Malvern, Rouge",NaN,NaN,NaN
1,1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek","M1C, Scarborough, Rouge Hill, Port Union, High...",NaN,NaN,NaN
2,2,M1E,Scarborough,"Guildwood, Morningside, West Hill","M1E, Scarborough, Guildwood, Morningside, West...",NaN,NaN,NaN
3,3,M1G,Scarborough,Woburn,"M1G, Scarborough, Woburn","Woburn, Scarborough—Guildwood, Scarborough, To...",43.759824,-79.225291
4,4,M1H,Scarborough,Cedarbrae,"M1H, Scarborough, Cedarbrae",NaN,NaN,NaN


In [83]:
#Saving the geographical coordinates csv file in data2 dataframe
data2 = pd.read_csv("http://cocl.us/Geospatial_data")
data2.rename(columns={'Postal Code': 'PostalCode'}, inplace=True)
data2.head()

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


### Genrating the dataframe

In [84]:
#Merging the gtwi dataframes into one data3 dataframe
data3 = pd.merge(data2, data1, how='inner', on=None, left_on=None, right_on=None,
         left_index=False, right_index=False, sort=True,
         suffixes=('_x', '_y'), copy=True, indicator=False,
         validate=None)

data3.head()

,PostalCode,Latitude,Longitude,Unnamed: 0,Borough,Neighbourhood,address,Coordinates,latitude,longitude
0,M1B,43.806686,-79.194353,0,Scarborough,"Malvern, Rouge","M1B, Scarborough, Malvern, Rouge",NaN,NaN,NaN
1,M1C,43.784535,-79.160497,1,Scarborough,"Rouge Hill, Port Union, Highland Creek","M1C, Scarborough, Rouge Hill, Port Union, High...",NaN,NaN,NaN
2,M1E,43.763573,-79.188711,2,Scarborough,"Guildwood, Morningside, West Hill","M1E, Scarborough, Guildwood, Morningside, West...",NaN,NaN,NaN
3,M1G,43.770992,-79.216917,3,Scarborough,Woburn,"M1G, Scarborough, Woburn","Woburn, Scarborough—Guildwood, Scarborough, To...",43.759824,-79.225291
4,M1H,43.773136,-79.239476,4,Scarborough,Cedarbrae,"M1H, Scarborough, Cedarbrae",NaN,NaN,NaN


In [85]:
#rearranging the columns
cols = data1.columns.tolist()
cols

['Unnamed: 0',
 'PostalCode',
 'Borough',
 'Neighbourhood',
 'address',
 'Coordinates',
 'latitude',
 'longitude']

In [87]:
new_column_order = ['PostalCode',
 'Borough',
 'Neighbourhood',
 'Latitude',
 'Longitude']
new_column_order

['PostalCode', 'Borough', 'Neighbourhood', 'Latitude', 'Longitude']

In [93]:
#generate the required dataframe by rearranging the coulmn order
data3 = data3[new_column_order]
data3.head(12)

,PostalCode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"Kennedy Park, Ionview, East Birchmount Park",43.727929,-79.262029
7,M1L,Scarborough,"Golden Mile, Clairlea, Oakridge",43.711112,-79.284577
8,M1M,Scarborough,"Cliffside, Cliffcrest, Scarborough Village West",43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff, Cliffside West",43.692657,-79.264848


In [94]:
#Sorting the values so as to match the values in the eample dataframe
sorted_df = data3.sort_values([ 'Neighbourhood', 'Latitude'], ascending=[True, True])
sorted_df.reset_index(inplace=True)
sorted_cols =sorted_df.columns.tolist()
example_dataframe = sorted_df[new_column_order]
example_dataframe.head(12)

,PostalCode,Borough,Neighbourhood,Latitude,Longitude
0,M1S,Scarborough,Agincourt,43.794200,-79.262029
1,M8W,Etobicoke,"Alderwood, Long Branch",43.602414,-79.543484
2,M3H,NorthYork,"Bathurst Manor, Wilson Heights, Downsview North",43.754328,-79.442259
3,M2K,NorthYork,Bayview Village,43.786947,-79.385975
4,M5M,NorthYork,"Bedford Park, Lawrence Manor East",43.733283,-79.419750
5,M5E,DowntownToronto,Berczy Park,43.644771,-79.373306
6,M1N,Scarborough,"Birch Cliff, Cliffside West",43.692657,-79.264848
7,M6K,WestToronto,"Brockton, Parkdale Village, Exhibition Place",43.636847,-79.428191
8,M7Y,EastToronto,"Business reply mail Processing Centre, South C...",43.662744,-79.321558
9,M5V,DowntownToronto,"CN Tower, King and Spadina, Railway Lands, Har...",43.628947,-79.394420


In [92]:
print('Required Dataframe Generated.')

Required Dataframe Generated.
